In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
config = get_config()
_, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

# Load pretrained weights, change latest weight version accordingly
model_filename = get_weights_file_path(config, f"21")
state = torch.load(model_filename, map_location=device)
model.load_state_dict(state["model_state_dict"])

def translate_custom_text(model, tokenizer_src, tokenizer_tgt, text, max_len, device):
    model.eval()  # Set the model to evaluation mode

    # Preprocess the input text
    input_tokens = tokenizer_src.encode(text, return_tensors='pt').to(device)
    input_mask = (input_tokens != tokenizer_src.pad_token_id).unsqueeze(0).to(device)

    with torch.no_grad():
        model_out = greedy_decode(model, input_tokens, input_mask, tokenizer_src, tokenizer_tgt, max_len, device)
        model_out_text = tokenizer_tgt.decode(model_out.detach().cpu().numpy(), skip_special_tokens=True)

    return model_out_text

# Example usage
input_text = "Hello, how are you?"
max_len = 50  # Define max length for the generated sequence
translation = translate_custom_text(model, tokenizer_src, tokenizer_tgt, input_text, max_len, device)
print(f"Input: {input_text}")
print(f"Translation: {translation}")
